# Molecular Arithmetic with `umda` and KIDA

In this notebook, we'll train a machine learning model to predict the rate coefficient of a reaction using the KIDA dataset, and `mol2vec` embeddings produced in the `umda` project.

In [1]:
import numpy as np
import pandas as pd
from subprocess import Popen, PIPE
from tempfile import NamedTemporaryFile
from rdkit import Chem
from umda import EmbeddingModel
from umda import compute
import re
from joblib import parallel_backend, load

Traceback (most recent call last):
  File "/home/kelvin/anaconda3/envs/rdkit/lib/python3.6/site-packages/rdkit/Chem/PandasTools.py", line 130, in <module>
    if 'display.width' in pd.core.config._registered_options:
AttributeError: module 'pandas.core' has no attribute 'config'


## Preprocessing

First we need to get the mapping from KIDA molecule (standard naming) to SMILES. The reaction network specification is completely in the "standard" naming convention, and so we'll need to do some homogenizing.

In [2]:
def parse_kida_format(path: str):
    data = list()
    with open(path) as read_file:
        for line in read_file.readlines():
            if not line.startswith("!"):
                split_line = line.split()
                react1, react2 = split_line[:2]
                # this regex is complicated because sometimes people
                # provide inconsistent formatting on the exponent
                values = re.findall(r"\d.\d{3,4}[eE][\+\-]\d{1,2}", line)
                alpha, beta, gamma = [float(value) for value in values]
                # grab the second integer specification, which is the reaction type
                integers = re.findall(r"\s+\d{1,2}\s+", line)
                react_index = int(integers[1])
                min_temp, max_temp = float(split_line[-6]), float(split_line[-5])
                data.append([react1, react2, alpha, beta, gamma, react_index, min_temp, max_temp])
    return data


def kida_to_dataframe(data, ignore=["CRP", "CR", "Photon", "GRAIN0", "GRAIN-", "XH"], react_class=[3, 4, 5]) -> pd.DataFrame:
    df = pd.DataFrame(data, columns=["A", "B", "alpha", "beta", "gamma", "react_class", "min_temp", "max_temp"])
    # ignore reactions with reactants that are in the ignore list, and ensure that
    # we have a mapping for the reaction class
    filtered_df = df.loc[
        (~df["B"].str.contains(rf"\b(?:{'|'.join(ignore)})\b")) & (~df["A"].str.contains(rf"\b(?:{'|'.join(ignore)})\b")) & (df["react_class"].isin(react_class))
    ]
    filtered_df.reset_index(drop=True, inplace=True)
    return filtered_df

In [3]:
kida_mols = pd.read_csv("../../data/external/kida-molecules_05_Jul_2020.csv").dropna()

In [4]:
kida_mols = kida_mols[kida_mols["InChI"] != "InChI="]

In [5]:
kida_mols.reset_index(drop=True, inplace=True)

In [6]:
mapping = dict()
with open("kida.inchi", "w+") as durr:
    for index, row in kida_mols.iterrows():
        inchi = row["InChI"]
        if "InChI" not in inchi:
            inchi = f"InchI={inchi.replace('Inchi', '')}"
        try:
            smi = Chem.MolToSmiles(Chem.MolFromInchi(inchi, sanitize=False), canonical=True)
            mapping[row["Formula"]] = smi
        except:
            print(f"{inchi} was untranscribable.")

InchI=1S/C3H4O/c1-2-3-4/h2-3H,1H2 was untranscribable.
InChI=1S/C7H/c1-3-5-7-6-4-2/h1H/Q+1 was untranscribable.
InchI=1S/C3H6O/c1-2-3-4/h3H,2H2,1H3 was untranscribable.
InchI=1S/C3HO/c1-2-3-4/h1H was untranscribable.
InchI==1S/HN/h1H/i1D was untranscribable.


In [7]:
kida_mols["SMILES"] = kida_mols["InChI"].map(mapping)

### Load in the KIDA reactions

In [8]:
reactions = parse_kida_format("../../data/external/kida.uva.2014/kida.uva.2014.dat")
reactions_df = kida_to_dataframe(reactions)

In [9]:
# vectorize the function
compute_rate = np.vectorize(compute.compute_rate)

In [10]:
# convert the standard formulae into SMILES
reactions_df["A_smi"] = reactions_df["A"].map(mapping)
reactions_df["B_smi"] = reactions_df["B"].map(mapping)

In [11]:
valid_reactions = reactions_df.dropna()
valid_reactions.reset_index(drop=True, inplace=True)

In [12]:
# load in the EmbeddingModel class predumped
embedder = load("../../models/EmbeddingModel.pkl")

### Generate all the kinetic data, and put it into a combined table

In [19]:
X, y = list(), list()
for index, row in valid_reactions.iterrows():
    min_temp, max_temp = row["min_temp"], row["max_temp"]
    # force the temperature ranges to be within [0, 300]
    min_temp = max(0., min_temp)
    max_temp = min(300., max_temp)
    temperatures = np.linspace(min_temp, max_temp, 30)
    rates = compute_rate(row["react_class"], temperatures, row["alpha"], row["beta"], row["gamma"])
    A, B = embedder.vectorize(row["A_smi"])[0], embedder.vectorize(row["B_smi"])[0]
    for rate, temp in zip(rates, temperatures):
        X.append(
            np.concatenate((A, B, [temp]))
        )
        y.append(rate)

In [21]:
# stack up all the embeddings into arrays
# A_emb = np.vstack([embedder.vectorize(smi)[0] for smi in valid_reactions["A_smi"].values])
# B_emb = np.vstack([embedder.vectorize(smi)[0] for smi in valid_reactions["B_smi"].values])

In [76]:
# combined = np.hstack([A_emb, B_emb])

In [22]:
# The nominal product given as the row-wise sum of A + B
# product = A_emb + B_emb

In [47]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, GridSearchCV

In [80]:
model = GradientBoostingRegressor(learning_rate=0.01, max_depth=3)

In [79]:
rates = valid_reactions["Rate"].values

In [81]:
model.fit(combined, rates)

GradientBoostingRegressor(learning_rate=0.01)

In [82]:
mean_squared_error(model.predict(combined), rates)

1.2584585280825188e-16

## K-Fold cross-validation for model selection

In [77]:
kfold = KFold(5, random_state=42, shuffle=True)

In [62]:
hyperparams = {"learning_rate": 10**np.arange(-4., 0., 1.), "n_estimators": [20, 50, 100, 150,], "max_depth": [1, 3, 5]}

In [63]:
search = GridSearchCV(
    GradientBoostingRegressor(), hyperparams, cv=kfold, scoring="neg_mean_squared_error"
)

In [64]:
with parallel_backend("threading", n_jobs=4):
    result = search.fit(combined, rates)

In [70]:
herp = result.cv_results_
del herp["params"]

In [72]:
pd.DataFrame(herp).sort_values(["rank_test_score"])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
49,19.016345,0.043406,0.001675,0.000067,0.1,1,200,-1.452442e-16,-1.720904e-16,-2.134558e-16,-1.407283e-16,-1.077960e-16,-1.558629e-16,3.530581e-17,1
48,14.225563,0.016151,0.001445,0.000036,0.1,1,150,-1.485245e-16,-1.744836e-16,-2.174884e-16,-1.428173e-16,-1.101626e-16,-1.586953e-16,3.582425e-17,2
39,55.240471,0.106077,0.003335,0.000429,0.01,3,200,-1.451457e-16,-1.681308e-16,-2.294617e-16,-1.434882e-16,-1.124063e-16,-1.597265e-16,3.911297e-17,3
51,13.774481,0.062657,0.001413,0.000031,0.1,3,50,-1.394676e-16,-1.614186e-16,-2.426668e-16,-1.427803e-16,-1.156558e-16,-1.603978e-16,4.363376e-17,4
52,27.592522,0.062880,0.001980,0.000087,0.1,3,100,-1.414791e-16,-1.519908e-16,-2.452125e-16,-1.483225e-16,-1.162011e-16,-1.606412e-16,4.409305e-17,5
44,87.984966,0.626240,0.005129,0.000364,0.01,5,200,-1.483535e-16,-1.605268e-16,-2.283187e-16,-1.446069e-16,-1.270674e-16,-1.617747e-16,3.495459e-17,6
47,9.505022,0.014821,0.001246,0.000031,0.1,1,100,-1.532548e-16,-1.782131e-16,-2.205125e-16,-1.456494e-16,-1.126009e-16,-1.620461e-16,3.597405e-17,7
38,41.701038,0.219038,0.002594,0.000135,0.01,3,150,-1.494284e-16,-1.722709e-16,-2.291165e-16,-1.458652e-16,-1.136837e-16,-1.620729e-16,3.837430e-17,8
42,44.309738,0.404892,0.002755,0.000148,0.01,5,100,-1.504285e-16,-1.701246e-16,-2.229229e-16,-1.465152e-16,-1.203805e-16,-1.620743e-16,3.430291e-17,9
43,66.323920,0.492083,0.003770,0.000071,0.01,5,150,-1.489823e-16,-1.649973e-16,-2.265249e-16,-1.461883e-16,-1.238710e-16,-1.621128e-16,3.477261e-17,10
